#NER workshop

In [1]:
!pip install datasets transformers seqeval

     |████████████████████████████████| 163kB 5.7MB/s 
     |████████████████████████████████| 1.8MB 36.0MB/s 
     |████████████████████████████████| 51kB 8.8MB/s 
     |████████████████████████████████| 245kB 55.9MB/s 
     |████████████████████████████████| 20.7MB 83.5MB/s 
     |████████████████████████████████| 890kB 56.0MB/s 
     |████████████████████████████████| 3.2MB 51.5MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=9a38c9a428975d064e6d169fb94b23d17d6b285ac456e4b1915ff3bd959a9d7e
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=015b75af9b1a8c4b166fa8e6a5a8474e7b07541ddc9a257559f3b3be6818e879
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses
  Found existing installation: pyarrow 0.14.

## Подготовка данных

In [2]:
from datasets import load_dataset, load_metric

In [3]:
datasets = load_dataset("conll2003")

Dataset conll2003 downloaded and prepared to /root/.cache/huggingface/datasets/conll2003/conll2003/1.0.0/63ba56944e35c1943434322a07ceefd79864672041b7834583709af4a5de4664. Subsequent calls will reuse this data.


In [4]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
datasets["train"][0]

{'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0],
 'id': '0',
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7],
 'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.']}

In [6]:
datasets["train"].features[f"ner_tags"]

Sequence(feature=ClassLabel(num_classes=9, names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], names_file=None, id=None), length=-1, id=None)

Таким образом, для тегов NER 0 соответствует 'O', 1 - 'B-PER' и т.д. Есть четыре NER класса, каждый из которых имеет префикс 'B-' (для начала) или 'I-' (для промежуточного уровня), которые указывают, является ли токен первым для текущей группы или нет:
- «PER» для человека
- ORG для организации
- "LOC" для местоположения
- «MISC» для разного

In [7]:
label_list = datasets["train"].features[f"ner_tags"].feature.names
label_list

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

In [8]:
MODEL_NAME = "distilbert-base-uncased"

In [9]:
from transformers import AutoTokenizer, AutoModel
    
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

### Токенизация

In [10]:
example = datasets["train"][4]
print(example["tokens"])

['Germany', "'s", 'representative', 'to', 'the', 'European', 'Union', "'s", 'veterinary', 'committee', 'Werner', 'Zwingmann', 'said', 'on', 'Wednesday', 'consumers', 'should', 'buy', 'sheepmeat', 'from', 'countries', 'other', 'than', 'Britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.']


In [11]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'germany', "'", 's', 'representative', 'to', 'the', 'european', 'union', "'", 's', 'veterinary', 'committee', 'werner', 'z', '##wing', '##mann', 'said', 'on', 'wednesday', 'consumers', 'should', 'buy', 'sheep', '##me', '##at', 'from', 'countries', 'other', 'than', 'britain', 'until', 'the', 'scientific', 'advice', 'was', 'clearer', '.', '[SEP]']


Здесь слова «Zwingmann» и «sheepmeat» были разделены на три подтокена.

Это означает, что нам нужно выполнить некоторую обработку наших меток, поскольку input ids, возвращаемые токенизатором, длиннее, чем списки меток, которые содержит наш набор данных, во-первых, и потому что могут быть добавлены некоторые специальные токены (мы можем использовать `[CLS]` и `[SEP]` выше).

In [12]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"ner_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

39 39


Мы устанавливаем метки всех специальных токенов -100 (индекс, который игнорируется PyTorch), а метки всех других токенов - на метку слова, из которого они происходят. Другая стратегия состоит в том, чтобы установить метку только для первого токена, полученного из данного слова, и присвоить метку -100 другим под токенам из того же слова. 

In [13]:
label_all_tokens = True

In [14]:
def tokenize_and_align_labels(examples, max_length: int, with_labels: bool = True):
    batch_length = min(max_length, max([len(example["tokens"]) for example in examples]))
    tokenized_inputs = tokenizer([example["tokens"] for example in examples], truncation=True, is_split_into_words=True, max_length=batch_length+2, padding="max_length")
    labels = []
    if all("ner_tags" in example for example in examples):
        for i, label in enumerate([example["ner_tags"] for example in examples]):
            word_ids = tokenized_inputs.word_ids(batch_index=i)
            previous_word_idx = None
            label_ids = []
            for word_idx in word_ids:
                # Special tokens have a word id that is None. We set the label to -100 so they are automatically
                # ignored in the loss function.
                if word_idx is None:
                    label_ids.append(-100)
                # We set the label for the first token of each word.
                elif word_idx != previous_word_idx:
                    label_ids.append(label[word_idx])
                # For the other tokens in a word, we set the label to either the current label or -100, depending on
                # the label_all_tokens flag.
                else:
                    label_ids.append(label[word_idx] if label_all_tokens else -100)
                previous_word_idx = word_idx

            labels.append(label_ids)
        if labels:
          tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [15]:
tokenize_and_align_labels(list(datasets['train'])[:3], 10)

{'input_ids': [[101, 7327, 19164, 2446, 2655, 2000, 17757, 2329, 12559, 1012, 102], [101, 2848, 13934, 102, 0, 0, 0, 0, 0, 0, 0], [101, 9371, 2727, 1011, 5511, 1011, 2570, 102, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]], 'labels': [[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100], [-100, 1, 2, -100, -100, -100, -100, -100, -100, -100, -100], [-100, 5, 0, 0, 0, 0, 0, -100, -100, -100, -100]]}

## Инициализация архитектуры

In [16]:
from typing import Dict
MAX_INPUT_LENGTH = 200

def generate_batches(input_data, max_length = MAX_INPUT_LENGTH):
    # return (text, sequence_length), labels
    tokenized_input = tokenize_and_align_labels(input_data, max_length)
    for key in tokenized_input.keys():
        tokenized_input[key] = torch.tensor(tokenized_input[key])
    return tokenized_input

In [17]:
import torch.nn as nn

class NERTransformer(nn.Module):
    def __init__(self, transformer, dropout, num_labels):
        super().__init__()

        self._num_labels = num_labels
        self._transformer = transformer
        self._transformer_output_size = (
            self._transformer.config.emb_dim
            if hasattr(self._transformer.config, "emb_dim")
            else self._transformer.config.hidden_size
        )
        self._head_dropout = nn.Dropout(dropout)
        self._classification_head = nn.Linear(
            self._transformer_output_size,
            self._num_labels
        )
        
        self._loss = nn.CrossEntropyLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        transformer_output = self._transformer(input_ids, attention_mask=attention_mask)[0]
        transformer_output = self._head_dropout(transformer_output)
        logits = self._classification_head(transformer_output)

        loss = None
        if labels is not None:
            # Only keep active parts of the loss
            if attention_mask is not None:
                active_loss = attention_mask.view(-1) == 1
                active_logits = logits.view(-1, self._num_labels)
                active_labels = torch.where(
                    active_loss, labels.view(-1), torch.tensor(self._loss.ignore_index).type_as(labels)
                )
                loss = self._loss(active_logits, active_labels)
            else:
                loss = self._loss(logits.view(-1, self._num_labels), labels.view(-1))

        return (loss, logits) if loss is not None else logits


In [18]:
import random
import numpy
import torch

def set_seed(seed: int, n_gpu: int):
    random.seed(seed)
    numpy.random.seed(seed)
    torch.manual_seed(seed)
    if n_gpu > 0:
        torch.cuda.manual_seed_all(seed)

set_seed(42, 1)

In [19]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda:0")
transformer = AutoModel.from_pretrained(MODEL_NAME)
model = NERTransformer(
    transformer=transformer,
    dropout=0.1,
    num_labels=len(label_list)
).to(device)

In [20]:
metric = load_metric("seqeval")

Будем использовать реализацию метрики из dataset библиотеки.

Нам нужно будет сделать небольшую постобработку наших прогнозов:
- выберите прогнозируемый индекс (с максимальным логитом) для каждого токена
- преобразовать его в его строковую метку
- игнорировать все где мы установили метку -100

In [21]:


def compute_metrics(predictions, labels):
    predictions = predictions.argmax(axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [22]:
labels = [label_list[i] for i in example[f"ner_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'LOC': {'f1': 1.0, 'number': 2, 'precision': 1.0, 'recall': 1.0},
 'ORG': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'PER': {'f1': 1.0, 'number': 1, 'precision': 1.0, 'recall': 1.0},
 'overall_accuracy': 1.0,
 'overall_f1': 1.0,
 'overall_precision': 1.0,
 'overall_recall': 1.0}

## Обучение модели

In [23]:
BATCH_SIZE=160

In [24]:
from torch.utils.data import DataLoader

def train(input_data, collate_fn, shuffle=True):
    model.train()
    # Train the model
    train_loss = 0
    n_batches = 0
    train_metrics = {"precision": 0, "recall": 0, "f1": 0, "accuracy": 0}
    data = DataLoader(input_data, batch_size=BATCH_SIZE, shuffle=shuffle, collate_fn=collate_fn)
    for i, batch in enumerate(data):
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
        loss, predictions = model(input_ids, attention_mask, labels)
        batch_metrics = compute_metrics(predictions, labels)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        for metric_name in train_metrics.keys():
            train_metrics[metric_name] += batch_metrics[metric_name]
        n_batches += 1

    for metric_name in train_metrics.keys():
        train_metrics[metric_name] /= n_batches
    return train_loss / n_batches, train_metrics

def test(input_data, collate_fn):
    test_loss = 0
    n_batches = 0
    model.eval()
    test_metrics = {"precision": 0, "recall": 0, "f1": 0, "accuracy": 0}
    data = DataLoader(input_data, batch_size=BATCH_SIZE, collate_fn=collate_fn)
    for i, batch in enumerate(data):
        input_ids, attention_mask, labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["labels"].to(device)
        with torch.no_grad():
            loss, predictions = model(input_ids, attention_mask, labels)
            batch_metrics = compute_metrics(predictions, labels)
            test_loss += loss.item()
            for metric_name in test_metrics.keys():
                test_metrics[metric_name] += batch_metrics[metric_name]
        n_batches += 1

    for metric_name in test_metrics.keys():
        test_metrics[metric_name] /= n_batches
    return test_loss / n_batches, test_metrics

In [25]:
import time
from torch.utils.data.dataset import random_split

N_EPOCHS = 2
min_valid_loss = float('inf')

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(datasets['train'], generate_batches)
    valid_loss, valid_acc = test(datasets['validation'], generate_batches)

    secs = int(time.time() - start_time)

    print(f"Epoch: {(epoch + 1)} | time in {secs} seconds")
    print(f"Loss: {train_loss:.4f}(train)\t|\tAcc: {train_acc}%(train)")
    print(f"Loss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc}%(valid)")

/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1 | time in 120 seconds
Loss: 0.2916(train)	|	Acc: {'precision': 0.6424790634868132, 'recall': 0.6549741525063043, 'f1': 0.6458714313258298, 'accuracy': 0.9177824251626585}%(train)
Loss: 0.0740(valid)	|	Acc: {'precision': 0.8966273344909659, 'recall': 0.9074875223023007, 'f1': 0.9018129001436348, 'accuracy': 0.9788309283418803}%(valid)
Epoch: 2 | time in 120 seconds
Loss: 0.0563(train)	|	Acc: {'precision': 0.9204179842698746, 'recall': 0.9281134211438016, 'f1': 0.9242055119187458, 'accuracy': 0.9840307543317937}%(train)
Loss: 0.0651(valid)	|	Acc: {'precision': 0.9019544774236459, 'recall': 0.9156965920853507, 'f1': 0.9086744694038265, 'accuracy': 0.9803676628475405}%(valid)


In [26]:
test_loss, test_acc = test(datasets["test"], generate_batches)

In [27]:
test_acc

{'accuracy': 0.9703612912062728,
 'f1': 0.8695913656356633,
 'precision': 0.8630696499392009,
 'recall': 0.8764931050115787}

In [28]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Предсказания модели

In [116]:
from typing import List
from nltk import word_tokenize


def model_prediction(texts: List[str]):
    prepared_samples = [{"tokens": word_tokenize(text)} for text in texts]
    tokenized_input = generate_batches(prepared_samples)
    input_ids, attention_mask = tokenized_input["input_ids"].to(device), tokenized_input["attention_mask"].to(device)
    model.eval()
    with torch.no_grad():
        predictions = model(input_ids, attention_mask)
    predicted_labels = []
    prediction_labels = predictions.argmax(axis=2).tolist()
    for sample_index, sample_predictions in enumerate(prediction_labels):
        predicted_labels.append([{tokenizer.convert_ids_to_tokens(input_ids[sample_index][token_index].item()): label_list[label_index]} for token_index, label_index in enumerate(sample_predictions)])
    return predicted_labels


In [117]:
texts = ["I like to work at Apple in California.", "The US audio-only social media app Clubhouse said it is reviewing its data protection practices after a report by the Stanford Internet Observatory said it contained security flaws that left users' data vulnerable to access by the Chinese government."]

In [120]:
predictions = model_prediction(texts)

In [121]:
predictions

[[{'[CLS]': 'O'},
  {'i': 'O'},
  {'like': 'O'},
  {'to': 'O'},
  {'work': 'O'},
  {'at': 'O'},
  {'apple': 'B-ORG'},
  {'in': 'O'},
  {'california': 'B-LOC'},
  {'.': 'O'},
  {'[SEP]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'B-ORG'},
  {'[PAD]': 'B-LOC'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'B-ORG'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'},
  {'[PAD]': 'O'}],
 [{'[CLS]': 'O'},
  {'the': 'O'},
  {'us': 'B-LOC'},
  {'audio': 'O'},
  {'-': 'O'},
  {'only': 'O'},
  {'social': 'O'},
  {'media': 'O'},
  {'app': 'O'},
  {'clubhouse': 'O'},
  {'said': 'O'},
  {'it': 'O'},
  {'is': '